## Chapter 10: The "Forest of Decisions"

Our next model will be the Random Forest. If Logistic Regression is like a judge weighing evidence on a scale, a Random Forest is like a committee of experts asking a series of "Yes/No" questions.
### 10.1 Why move to Random Forest?

Logistic Regression has a "Linear Constraint." It assumes that if a feature is important, its impact is consistent. But survival on the Titanic was full of interactions:

    Interaction Example: Being a child was good for survival, but being a child in 3rd Class was very different from being a child in 1st Class.

    The Random Forest Solution: It can create branches like: If Pclass is 3 AND Age < 10, then check Fare.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Load the data we cleaned in the first notebook
df = pd.read_csv('../data/titanic_model_ready.csv')

# 2. Prepare X and y
X = df.drop('Survived', axis=1)
y = df['Survived']

# 3. The "Golden Split" (Using 42 again so results are comparable)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Ready to train! Training on {len(X_train)} samples, testing on {len(X_test)}.")

Ready to train! Training on 712 samples, testing on 179.


A Random Forest is just a collection of Decision Trees. To build the forest, we first have to build a single tree that can think for itself.

### 1.1 The Logic: What is "Gini Impurity"?

In Logistic Regression, we used Log Loss to measure error. In Decision Trees, we use Gini Impurity. It measures how "mixed" a group is.

*   **Pure Group:** 100% Survivors (Gini = 0).
*   **Mixed Group:** 50% Survivors, 50% Died (Gini = 0.5 — This is "Chaos").

The goal of our tree is to ask questions that reduce the Chaos the most.

### 1.2 The Building Blocks (The "Node")

First, we need a structure to hold our questions.


In [23]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature       # Which column are we looking at? (e.g., Sex)
        self.threshold = threshold   # At what value do we split? (e.g., 0.5)
        self.left = left             # The "Yes" branch
        self.right = right           # The "No" branch
        self.value = value           # If it's a leaf, what's the answer? (Survived/Died)

### The "Chaos" Math ($1 - \sum p^2$)

This is the Gini Impurity formula. It measures how often a randomly chosen element from the set would be incorrectly labeled.

$$ Gini = 1 - (p_{died}^2 + p_{survived}^2) $$


In [24]:
def calculate_gini(y):
    m = len(y)
    if m == 0: return 0
    p1 = np.sum(y) / m  # Probability of surviving
    p0 = 1 - p1         # Probability of dying
    return 1 - (p0**2 + p1**2)

In [ ]:
def best_split(X, y):
    best_gini = 1.0 # Initialize with worst Gini
    split_idx, split_thresh = None, None # winning Feature (column index) and the winning Threshold (the number we split on)
    
    for feature_idx in range(X.shape[1]): # loop through each feature/column
        thresholds = np.unique(X[:, feature_idx]) # get all unique values in this column
        for thresh in thresholds: # loop through each unique value
            # Try splitting the data here 
            left_idxs = np.where(X[:, feature_idx] <= thresh)[0] # if value is less than or equal to threshold
            right_idxs = np.where(X[:, feature_idx] > thresh)[0]
            
            if len(left_idxs) == 0 or len(right_idxs) == 0: continue
            
            # Calculate weighted Gini
            g_left = calculate_gini(y[left_idxs])
            g_right = calculate_gini(y[right_idxs])
            n, n_l, n_r = len(y), len(left_idxs), len(right_idxs)
            weighted_gini = (n_l/n) * g_left + (n_r/n) * g_right
            
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                split_idx = feature_idx
                split_thresh = thresh
                
    return split_idx, split_thresh

### 1.5 The Strategy for the Forest

Once we have a single tree working, the "Random Forest" part is actually quite simple:

    Bootstrapping: Give each tree a random 80% of the data.

    Feature Randomness: Each tree is only allowed to look at a few random features (e.g., it can't see "Sex" so it has to find other patterns).

    Voting: Ask all trees for their opinion and take the majority.

In [26]:
class ScratchDecisionTree:
    def __init__(self, max_depth=5):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))

        # Stopping Criteria:
        # 1. We reached max depth
        # 2. There is only one class left (Pure)
        # 3. No more samples
        if depth >= self.max_depth or num_labels == 1 or num_samples < 2:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Find the best split using the logic from our previous step
        feat_idx, thresh = best_split(X, y)

        if feat_idx is None:
            return Node(value=self._most_common_label(y))

        # Grow the branches
        left_idxs = np.where(X[:, feat_idx] <= thresh)[0]
        right_idxs = np.where(X[:, feat_idx] > thresh)[0]

        left = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(feature=feat_idx, threshold=thresh, left=left, right=right)

    def _most_common_label(self, y):
        return np.round(np.mean(y)) # For binary 0/1, the mean rounded is the majority
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, x, node):
        if node.value is not None:
                return node.value
    
        if x[node.feature] <= node.threshold:
                return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [27]:
# 1. Initialize and Train
my_tree = ScratchDecisionTree(max_depth=3)
my_tree.fit(X_train.values, y_train.values)

# 2. Predict on the Test Set
y_tree_pred = my_tree.predict(X_test.values)

# 3. Check Accuracy
tree_acc = np.mean(y_tree_pred == y_test.values)
print(f"Scratch Single Tree Accuracy: {tree_acc:.2%}")

Scratch Single Tree Accuracy: 79.89%


In [29]:
class ScratchRandomForest:
    def __init__(self, n_trees=10, max_depth=5, sample_sz=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_sz = sample_sz
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = X.shape[0]
        if self.sample_sz is None: self.sample_sz = n_samples
        
        for _ in range(self.n_trees):
            # 1. Create a Bootstrap Sample (Randomly pick rows with replacement)
            indices = np.random.choice(n_samples, self.sample_sz, replace=True)
            X_sample, y_sample = X[indices], y[indices]
            
            # 2. Train a tree on this random sample
            tree = ScratchDecisionTree(max_depth=self.max_depth)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        # 3. Get predictions from all trees
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        
        # 4. Majority Vote: Average the predictions and round to 0 or 1
        # (axis=0 means we average vertically across the trees)
        avg_preds = np.mean(tree_preds, axis=0)
        return (avg_preds >= 0.5).astype(int)

In [30]:
# Initialize our ensemble
my_forest = ScratchRandomForest(n_trees=10, max_depth=5)

# Train (This will take a few seconds because it builds 10 trees!)
my_forest.fit(X_train.values, y_train.values)

# Predict
y_forest_pred = my_forest.predict(X_test.values)

# Final Accuracy
forest_acc = np.mean(y_forest_pred == y_test.values.flatten())
print(f"Scratch Random Forest Accuracy: {forest_acc:.2%}")

Scratch Random Forest Accuracy: 78.21%


### 1.3 The Theory: What is a Random Forest?

Imagine you are trying to decide if a passenger survived.

*   **A Decision Tree asks:** "Is it a woman?" → "Yes" → "Is she in 1st class?" → "Yes" → Survived.
*   **A Random Forest** is a collection of 100 different trees. Each tree gets to see a slightly different version of the data.
*   **The Final Answer:** All 100 trees "vote." If 80 trees say "Survived" and 20 say "Died," the forest predicts Survived.

### 1.4 Training the "Forest"

Here is the code to kick off your new notebook's first model. We will start with a "Constrained Forest" (max_depth=5) to make sure it doesn't just memorize the data.


In [28]:
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred)*100, "%")

Accuracy: 79.88826815642457 %
